In [37]:
import tfFLAGS
tfFLAGS.LEARNING_RATE_DECAY_FACTOR = 0.1;           tfFLAGS.NUM_CLASSES = 10;       tfFLAGS.run_once = True
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000;   tfFLAGS.batch_size = 128;       tfFLAGS.use_fp16 = False
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000;    tfFLAGS.max_steps = 4000;      tfFLAGS.log_frequency = 100
tfFLAGS.MOVING_AVERAGE_DECAY = 0.9999;   tfFLAGS.NUM_EPOCHS_PER_DECAY = 350.0;      tfFLAGS.IMAGE_SIZE = 32;
tfFLAGS.INITIAL_LEARNING_RATE = 0.1;     tfFLAGS.eval_interval_secs = 60 * 5;       tfFLAGS.num_examples = 1000

In [2]:
import tensorflow as tf

In [31]:
conv1Size = 3; conv1Shape = [conv1Size, conv1Size, 3, 64]
conv2Size = 5; conv2Shape = [conv2Size, conv2Size, 64, 128]

pool1S = 2; pool1ksize=[1, pool1S, pool1S, 1]; pool1St = 2; pool1strides=[1, pool1St, pool1St, 1]; pool1padding='SAME'
pool2S = 2; pool2ksize=[1, pool2S, pool2S, 1]; pool2St = 2; pool2strides=[1, pool2St, pool2St, 1]; pool2padding='SAME'

local3InputDepth = 384; local3OutputDepth = 384
local4InputDepth = local3OutputDepth; local4OutputDepth = 192
softmax_linearInput = local4OutputDepth

In [32]:
print_('Network summary:')
print_('conv1Shape:', conv1Shape)
print_('pool1ksize:', pool1ksize, ' | pool1strides:', pool1strides, ' | pool1padding:', pool1padding)
print_('conv2Shape:', conv2Shape)
print_('pool2ksize:', pool2ksize, ' | pool2strides:', pool2strides, ' | pool2padding:', pool2padding)
print_('local3InputDepth:', local3InputDepth, ' | local3OutputDepth:', local3OutputDepth)
print_('local4InputDepth:', local4InputDepth, ' | local4OutputDepth:', local4OutputDepth)
print_('softmax_linearInput:', softmax_linearInput)
print_()
numParamConv1 = (conv1Shape[0] * conv1Shape[1] * conv1Shape[2] + 1) * conv1Shape[-1]
print_('Number of hidden parameters of conv1:', numParamConv1)
#print_('Number of hidden parameters of norm1:', numParamConv1 / pool1S**1)
numParamConv2 = (conv2Shape[0] * conv2Shape[1] * conv2Shape[2] + 1) * conv2Shape[-1]
print_('Number of hidden parameters of conv2:', numParamConv2)
numParamLocal3 = (local3InputDepth + 1) * local3OutputDepth
print_('Number of hidden parameters of local3:', numParamLocal3)
numParamLocal4 = (local4InputDepth + 1) * local4OutputDepth
print_('Number of hidden parameters of local4:', numParamLocal4)
numParamsoftmax = (softmax_linearInput + 1) * tfFLAGS.NUM_CLASSES
print_('Number of hidden parameters of softmax:', numParamsoftmax)
print_('Total number of hidden parameters:', numParamConv1 + numParamConv2 + numParamLocal3 + numParamLocal4 + numParamsoftmax)


Network summary:
conv1Shape: [3, 3, 3, 64]
pool1ksize: [1, 2, 2, 1]  | pool1strides: [1, 2, 2, 1]  | pool1padding: SAME
conv2Shape: [5, 5, 64, 128]
pool2ksize: [1, 2, 2, 1]  | pool2strides: [1, 2, 2, 1]  | pool2padding: SAME
local3InputDepth: 384  | local3OutputDepth: 384
local4InputDepth: 384  | local4OutputDepth: 192
softmax_linearInput: 192

Number of hidden parameters of conv1: 1792
Number of hidden parameters of conv2: 204928
Number of hidden parameters of local3: 147840
Number of hidden parameters of local4: 73920
Number of hidden parameters of softmax: 1930
Total number of hidden parameters: 430410


In [34]:
from MyModel import * 
def inferenceOnJupyter(images):
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = variable_with_weight_decay('weights', shape=conv1Shape, stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_on_cpu('biases', [conv1Shape[-1]], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        activation_summary(conv1)
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=pool1ksize, strides=pool1strides, padding=pool1padding, name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = variable_with_weight_decay('weights', shape=conv2Shape, stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_on_cpu('biases', [conv2Shape[-1]], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        activation_summary(conv2)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=pool2ksize, strides=pool2strides, padding=pool2padding, name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [tfFLAGS.batch_size, -1])
        local3InputDepth = reshape.get_shape()[1].value
        weights = variable_with_weight_decay('weights', shape=[local3InputDepth, local3OutputDepth], stddev=0.04, wd=0.004)
        biases = variable_on_cpu('biases', [local3OutputDepth], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)        
        activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = variable_with_weight_decay('weights', shape=[local3OutputDepth, local4OutputDepth], stddev=0.04, wd=0.004)
        biases = variable_on_cpu('biases', [local4OutputDepth], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        activation_summary(local4)
        
    with tf.variable_scope('softmax_linear') as scope:
        weights = variable_with_weight_decay('weights', [softmax_linearInput, tfFLAGS.NUM_CLASSES], stddev=1/192.0, wd=0.0)
        biases = variable_on_cpu('biases', [tfFLAGS.NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        activation_summary(softmax_linear)
    return softmax_linear

In [35]:
import MyModel
inferenceOnMyModel = MyModel.inference
MyModel.inference = inferenceOnJupyter

In [38]:
from runCIFAR10_Tensorflow import main
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-06 05:20:37.629658: Running on server...
The experiment details:
max_steps = 4000 log_frequency = 100 num_gpus = 2
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
2017-05-06 05:21:03.312783: step 0, loss = 10.28 (12.5 examples/sec; 10.261 sec/batch)
2017-05-06 05:21:24.767219: step 100, loss = 9.07 (1279.0 examples/sec; 0.100 sec/batch)
2017-05-06 05:21:44.548738: step 200, loss = 8.50 (1316.2 examples/sec; 0.097 sec/batch)
2017-05-06 05:22:05.408034: step 300, loss = 7.86 (1318.2 examples/sec; 0.097 sec/batch)
2017-05-06 05:22:25.192726: step 400, loss = 7.34 (1322.5 examples/s